Instalações

In [1]:
from sklearn.preprocessing import OneHotEncoder, normalize
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import silhouette_score, accuracy_score
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import os
import csv
from sklearn.metrics.pairwise import pairwise_distances_argmin_min
import requests

Essa função é usada para a gambiarra de baixar as imagens

In [2]:
def baixar_imagem(url, nome_arquivo, id):
    try:
        # Faz uma requisição GET para a URL da imagem
        resposta = requests.get(url)

        # Verifica se a requisição foi bem-sucedida
        if resposta.status_code == 200:
            # Abre o arquivo no modo de escrita binária ('wb') e escreve o conteúdo da resposta
            with open(f'{nome_arquivo}{id}.jpg', 'wb') as arquivo:
                arquivo.write(resposta.content)
            print(f"Imagem salva como {nome_arquivo}")
        else:
            print("Falha ao baixar a imagem:", resposta.status_code)
    except Exception as e:
        print("Erro ao baixar a imagem:", str(e))

# Exemplo de uso
url = "http://assets.myntassets.com/v1/images/style/properties/3c7a1a7a39a32e91474d905ca3090bf1_images.jpg"
nome_arquivo = "C:/Users/eliab/Documents/projetao/"
baixar_imagem(url, nome_arquivo, 1)

Imagem salva como C:/Users/eliab/Documents/projetao/


### FUNÇÃO MUITO IMPORTANTE
Image_to_pixels -> Função que redimensiona e converte as imagens em um vetor de pixels.
Esta função é chamada no momento de estruturar os dados, na variável X(onde estarão todos os nossos dados) que aparece duas celulas a baixo.
Paraca imagem que será usada no treinamento, esta função será chamada uma vez, e recebe como parâmetro a própria imagem.

In [2]:
def image_to_pixels(image):


    # Redimensionar a imagem
    image = cv2.resize(image, (180, 180))

    # Converter para um array numpy
    pixels = image.flatten()
    pixels_array = np.array(pixels)

    #normalizando os pixels para um intervalo (-1,1)
    normalized_pixels = normalize([pixels_array], norm='l2', axis=1)

    return normalized_pixels[0]

Essa função é usada para a gambiarra de baixar as imagens

In [4]:
def separar_ids_links(arquivo_csv):
    ids = []
    links = []
    with open(arquivo_csv, 'r', newline='', encoding='utf-8') as csv_file:
        leitor_csv = csv.reader(csv_file)
        cont = 0
        for linha in leitor_csv:
            #ids.append(linha[0])
            if cont < 4000:
              cont +=1
              baixar_imagem(linha[1], "C:/Users/eliab/Documents/projetao/Roupas_images/", linha[0][:-4])
            else:
              break


# Exemplo de uso
arquivo_csv = 'images.csv'
separar_ids_links(arquivo_csv)

Imagem salva como C:/Users/eliab/Documents/projetao/Roupas_images/
Imagem salva como C:/Users/eliab/Documents/projetao/Roupas_images/
Imagem salva como C:/Users/eliab/Documents/projetao/Roupas_images/
Imagem salva como C:/Users/eliab/Documents/projetao/Roupas_images/


KeyboardInterrupt: 


### JUNTANDO AS IMAGENS NO FORMATO IDEAL PARA O TREINAMENTO
Pegando as imagens, redimensionando e convertendo em um vetor de pixels normalizado(-1,1)


In [3]:
X = []

# Diretório que contém as imagens
diretorio = "C:/Users/eliab/Documents/projetao/Roupas_images"  # Substitua pelo caminho do seu diretório

# Iterar sobre os arquivos no diretório
ids = [] #id[0] = id da imagem 0 em X
for nome_arquivo in os.listdir(diretorio):
    caminho_imagem = os.path.join(diretorio, nome_arquivo)
    if os.path.isfile(caminho_imagem):  # Verificar se o caminho é um arquivo
        imagem = cv2.imread(caminho_imagem)
        X.append(image_to_pixels(imagem))
        ids.append(nome_arquivo[:-4])
X = np.array(X)
print(X)
print(ids)

[[0.00378868 0.00378868 0.00378868 ... 0.00378868 0.00378868 0.00378868]
 [0.00372292 0.00372292 0.00372292 ... 0.00372292 0.00372292 0.00372292]
 [0.00362749 0.00362749 0.00362749 ... 0.00362749 0.00362749 0.00362749]
 ...
 [0.00372072 0.00372072 0.00372072 ... 0.00372072 0.00372072 0.00372072]
 [0.00360337 0.00360337 0.00360337 ... 0.00360337 0.00360337 0.00360337]
 [0.00351054 0.00351054 0.00351054 ... 0.00351054 0.00351054 0.00351054]]
['27277', '27279', '27283', '27284', '27410', '27419', '27421', '27426', '27428', '27614', '27622', '27841', '27846', '27848', '27870', '27877', '27879', '27883', '27884', '28003', '28004', '28032', '28035', '28200', '28207', '28231', '28236', '28238', '28451', '28456', '28458', '28460', '28467', '28469', '28652', '28655', '28690', '28697', '28699', '28831', '28836', '29113', '29114', '29122', '29125', '29317', '29319', '29321', '29546', '29548', '29570', '29583', '29584', '29742', '29745', '29773', '29774', '29780', '29787', '29789', '29910', '29917

### ALINHANDO OS IDS DAS IMAGES.CSV COM OS IDS DE STYLES.CSV PARA QUE CADA DESCRIÇÃO CORRESPONDA A SUA RESPECTIVA IMAGEM NA HORA DE TREINAR

Copiando as labels e criando um novo dataframe que guar os rotulos dos dados

In [4]:
y_labels = []
arquivo_csv = "styles.csv"
with open(arquivo_csv, 'r', newline='', encoding='utf-8') as csv_file:
        leitor_csv = csv.reader(csv_file)
        for linha in leitor_csv:
            linha_id = linha[0]
            for id in ids:
                if linha_id == id:
                    y_labels.append(linha)

#ordenando as labels para combinarem com suas respectivas imagens
y_labels = sorted(y_labels, key=lambda x: x[0])
print(y_labels)


[['27277', 'Women', 'Apparel', 'Topwear', 'Kurtas', 'Green', 'Fall', '2011', 'Ethnic'], ['27279', 'Women', 'Apparel', 'Topwear', 'Kurtas', 'Blue', 'Fall', '2011', 'Ethnic'], ['27283', 'Women', 'Apparel', 'Topwear', 'Kurtas', 'Magenta', 'Fall', '2011', 'Ethnic'], ['27284', 'Women', 'Apparel', 'Topwear', 'Kurtas', 'Brown', 'Fall', '2011', 'Ethnic'], ['27410', 'Women', 'Apparel', 'Loungewear and Nightwear', 'Lounge Pants', 'Multi', 'Summer', '2018', 'Casual'], ['27419', 'Women', 'Apparel', 'Loungewear and Nightwear', 'Lounge Pants', 'Black', 'Winter', '2015', 'Casual'], ['27421', 'Women', 'Apparel', 'Loungewear and Nightwear', 'Lounge Pants', 'Blue', 'Winter', '2015', 'Casual'], ['27426', 'Women', 'Apparel', 'Bottomwear', 'Shorts', 'Grey Melange', 'Winter', '2015', 'Casual'], ['27428', 'Women', 'Apparel', 'Topwear', 'Tshirts', 'Black', 'Fall', '2015', 'Casual'], ['27614', 'Men', 'Apparel', 'Bottomwear', 'Trousers', 'Beige', 'Summer', '2012', 'Casual'], ['27622', 'Men', 'Apparel', 'Bottomw

### TRANFORMANDO OS LABELS EM COLUNAS CODIFICADAS
Esta célula converte os dados categóricos para um formato que o modelo pode compreender.
É nesta etapa que definimos qual coluna será o rótulo que queremos prever.

In [5]:
# Definindo os nomes das colunas
colunas = ["id", "gender", "masterCategory", "subCategory", "articleType", "baseColour", "season", "year", "usage"]

# Criando o DataFrame
df = pd.DataFrame(y_labels, columns=colunas)

# Selecionar apenas as colunas categóricas
dados_categoricos = df.drop(columns=["id", "gender", "subCategory", "masterCategory", "articleType", "baseColour", "season", "year", "usage"])


#COLUNA QUE VAI SER PREVISTA
coluna_codificada = colunas[1]


# Ajustar e transformar os dados categóricos
# Criar um codificador one-hot
encoder = LabelEncoder()
dados_categoricos[coluna_codificada] = encoder.fit_transform(df[coluna_codificada])
#print(dados_categoricos)




### Separando dados de treinamento e iniciando o modelo

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, dados_categoricos, test_size=0.2, random_state=42)
modelo = KNeighborsClassifier(n_neighbors=5, weights='distance')
modelo.fit(X_train, y_train)


C:\Users\eliab\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(weights='distance')

### Testes -> ainda iniciando

In [179]:
y_pred = modelo.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Acurácia do KNN com pesos personalizados:", accuracy)

Acurácia do KNN com pesos personalizados: 0.7471264367816092


In [7]:
img = cv2.imread("C:/Users/eliab/Documents/projetao/1.jpg")
img_processed = image_to_pixels(img)
predicted_label = modelo.predict([img_processed])
predicted_label = encoder.inverse_transform(predicted_label)
print(predicted_label)

['Men']


### Esta celula foi utilizada para apagar a última coluna.
Foi preciso apagar por que cada valor era uma frase e em alguns casos(quando havia uma virgula na frase) esbarrava em um erro ao tentar abrir o csv com pandas
Em segundo lugar aquele tipo de dados seria dificilmente bem processado pelo cluster

In [ ]:
novas_imagens = []
# Diretório que contém as imagens

# Iterar sobre os arquivos no diretório
imagem = cv2.imread("/content/test2.jpg")
imagem2 = cv2.imread("/content/teste1.jpg")


novas_imagens.append(image_to_pixels(imagem))
print(modelo.components_[1])
print(novas_imagens[0])
# Encontrar o cluster mais próximo da nova imagem
cluster_centroide_indices = pairwise_distances_argmin_min(novas_imagens, modelo.components_)[0]
cluster_centroide_indice = cluster_centroide_indices[0]

# Encontrar imagens semelhantes no cluster mais próximo
imagens_similares_indices = np.where(modelo.labels_ == cluster_centroide_indice)[0]

print("imgs ", imagens_similares_indices)